In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import  preprocessing, ensemble
from sklearn.metrics import log_loss,accuracy_score
from sklearn.cross_validation import KFold,StratifiedKFold
import re
import string
from collections import defaultdict, Counter
#import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

/opt/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import pickle
from mochi import *

In [12]:
#try xgboost
#original fucntion from SRK
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, \
     seed_val=0, early_stop = 20,num_rounds=10000, eta = 0.1,\
     max_depth = 6,cv_dict = None,verbose_eval=True):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = eta
    param['max_depth'] = max_depth
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.3
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y,feature_names=feature_names)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y,feature_names=feature_names)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist,\
        early_stopping_rounds=early_stop,evals_result = cv_dict,verbose_eval = verbose_eval)
    else:
        xgtest = xgb.DMatrix(test_X,feature_names=feature_names)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [3]:
#lodaing data
data_path = "../../kaggleData/2sigma/"
train_file = data_path + "train.json"
test_file = data_path + "test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)


(49352, 15)
(74659, 14)


In [4]:
#basic numerical features
features_to_use  = ["bathrooms", "bedrooms", "latitude", "longitude", "price"]

In [5]:
#some transfromed features
# count of photos #
train_df["num_photos"] = train_df["photos"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

# count of "features" #
train_df["num_features"] = train_df["features"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)

# count of words present in description column #
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

# convert the created column to datetime object so as to extract more features 
train_df["created"] = pd.to_datetime(train_df["created"])
test_df["created"] = pd.to_datetime(test_df["created"])

# Let us extract some features like year, month, day, hour from date columns #
train_df["created_year"] = train_df["created"].dt.year
test_df["created_year"] = test_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
test_df["created_month"] = test_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
test_df["created_day"] = test_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour
test_df["created_hour"] = test_df["created"].dt.hour

#some new numerical features related to the price
train_df["price_per_bath"] =  (train_df["price"]*1.0/train_df["bathrooms"]).replace(np.Inf,-1)
train_df["price_per_bed"] = (train_df["price"]*1.0/train_df["bedrooms"]).replace(np.Inf,-1)
train_df["bath_per_bed"] = (train_df["bathrooms"]*1.0/train_df["bedrooms"]).replace(np.Inf,-1)
train_df["price_per_room"] = (train_df["price"]*1.0/(train_df["bedrooms"]+train_df["bathrooms"])).replace(np.Inf,-1)

test_df["price_per_bath"] =  (test_df["price"]*1.0/test_df["bathrooms"]).replace(np.Inf,-1)
test_df["price_per_bed"] = (test_df["price"]*1.0/test_df["bedrooms"]).replace(np.Inf,-1)
test_df["bath_per_bed"] = (test_df["bathrooms"]*1.0/test_df["bedrooms"]).replace(np.Inf,-1)
test_df["price_per_room"] = (test_df["price"]*1.0/(test_df["bedrooms"]+test_df["bathrooms"])).replace(np.Inf,-1)


# adding all these new features to use list # "listing_id",
features_to_use.extend(["num_photos", "num_features", "num_description_words",\
                        "created_year","listing_id", "created_month", "created_day", "created_hour"])
#price new features
features_to_use.extend(["price_per_bed","bath_per_bed","price_per_room"])

#for latter use
train_df["dayofyear"] = train_df["created"].dt.dayofyear
test_df["dayofyear"] = test_df["created"].dt.dayofyear

In [6]:
processMap(train_df)
processMap(test_df)

#adding the house type
train_df['house_type']=map(lambda x,y:(x,y),train_df['bedrooms'],train_df['bathrooms'])
train_df['house_type'] = train_df['house_type'].apply(str)
#filling outliers with nan
test_df['house_type']=map(lambda x,y:(x,y),test_df['bedrooms'],test_df['bathrooms'])
test_df['house_type'] = test_df['house_type'].apply(str)

"""
new categorical data generated from the old ones
"""
#new feature for the street_address, use them instead of the original one
train_df["street_name"] = train_df["street_address"].apply(proecessStreet)
test_df["street_name"] = test_df["street_address"].apply(proecessStreet)

train_df['building0']=map(lambda x:1 if x== '0' else 0,train_df['building_id'])
test_df['building0']=map(lambda x:1 if x== '0' else 0,test_df['building_id'])

In [7]:
#dealing with features

#preprocessing for features
train_df["features"] = train_df["features"].apply(lambda x:["_".join(i.split(" ")).lower().strip().replace('-','_') \
                                                            for i in x])
test_df["features"] = test_df["features"].apply(lambda x:["_".join(i.split(" ")).lower().strip().replace('-','_')\
                                                          for i in x])
#create the accept list
accept_list = list(featureList(train_df,test_df,limit = 0.001))

#map the feature to dummy slots
featureMapping(train_df,test_df,accept_list)
features_to_use.extend(map(lambda x : 'with_'+x,accept_list))

In [8]:
#the basic features from preprocessing 
features = list(features_to_use)

#features to be added during cv by cv-manner statistics
features.extend(['manager_id_perf','house_type_perf'])
features.extend(['m3perf','m7perf','m14perf','m30perf'])
features.extend(['manager_id_nrank'])


#categorical features to be added
categorical = ["display_address", "street_address","street_name",'building_id',\
'manager_id','building0','house_type']
features.extend(categorical)
features.extend(['cluster_id_10','cluster_id_30'])

#statistical features
features.extend(['m_m_distance','mlon','mlat'])

main_st_nf = ["bathrooms", "bedrooms","price_per_bed","bath_per_bed",\
"price_per_room","num_photos", "num_features", "num_description_words",'price']
main_statistics =['mean','max','min','median']

for st in main_statistics:
    features.extend(map(lambda x : 'manager_id_'+x+'_'+st,main_st_nf))
    features.extend(map(lambda x : 'house_type_'+x+'_'+st,main_st_nf)) 

features.extend(map(lambda x : 'cluster_id_10_'+x+'_'+'mean',main_st_nf))
features.extend(map(lambda x : 'cluster_id_30_'+x+'_'+'mean',main_st_nf))

price_related = ['price_per_bed','price_per_room','price']

features.extend(['manager_id_size','house_type_size'])

In [9]:
    #=============================================================        
    """feature engineerings for the categorical features"""
    #fill substitute the small size values by their mean
    for f in categorical:    
        if train_df[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_df[f])+list(test_df[f]))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
    
    for f in categorical:
        train_df,test_df  = singleValueConvert(train_df,test_df,f,1)
    
    #kmeans grouping
    getCluster(train_df,test_df,30)
    getCluster(train_df,test_df,10)
    
    #K-FOLD evaluation for the statistic features
    skf=KFold(len(train_df['interest_level']),5,shuffle=True,random_state = 42)
    #dev set adding manager skill
    for train,test in skf:
            performance_eval(train_df.iloc[train,:],train_df.iloc[test,:],feature='manager_id',k=5,g=10,
                           update_df = train_df,smoothing=False)
            performance_eval(train_df.iloc[train,:],train_df.iloc[test,:],feature='house_type',k=5,g=10,
                           update_df = train_df,smoothing=False)
            temporalManagerPerf(train_df.iloc[train,:],train_df.iloc[test,:],update_df = train_df)

    performance_eval(train_df,test_df,feature='manager_id',k=5,g=10,smoothing=False)
    performance_eval(train_df,test_df,feature='house_type',k=5,g=10,smoothing=False)
    temporalManagerPerf(train_df,test_df)
        
        
    #statitstic
    for f in main_st_nf:
        #print f
        categorical_statistics(train_df,test_df,'manager_id',f)
        categorical_statistics(train_df,test_df,'cluster_id_10',f)
        categorical_statistics(train_df,test_df,'cluster_id_30',f)
        categorical_statistics(train_df,test_df,'house_type',f)
        categorical_size(train_df,test_df,'manager_id')
        categorical_size(train_df,test_df,'house_type')
    
    #for f in price_related:
    #    rank_on_categorical(train_df,test_df,'house_type_30',f,random =None)

    
    #manager main location
    manager_lon_lat(train_df,test_df)
    
    for f in categorical:
        if train_df[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_df[f])+list(test_df[f]))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))

In [14]:
"""output the processed dataframes"""
#shorten reprocessing time: save the preprocessed train_df and test_df with some basic features
train_df.to_json(data_path+'processed_train_df.json')
test_df.to_json(data_path+'processed_test_df.json')
#print features_to_use
#shorten reprocessing time: load the preprocessed train_df and test_df with some basic features

#train_df=pd.read_json(data_path+'processed_train_df.json')
#test_df=pd.read_json(data_path+'processed_test_df.json')


In [15]:
#output the feature list

xgb14featureFile = 'xgb145.feat'
fileObject = open(xgb14featureFile,'wb') 
pickle.dump(features,fileObject)   
fileObject.close()
"""
xgb14featureFile = 'xgb145.feat'
fileObject = open(xgb14featureFile,'r')
features = pickle.load(fileObject)
fileObject.close()
"""

"\nxgb14featureFile = 'xgb145.feat'\nfileObject = open(xgb14featureFile,'r')\nfeatures = pickle.load(fileObject)\nfileObject.close()\n"

In [10]:
#prepare for validation
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=StratifiedKFold(train_y,5,shuffle=True,random_state = 42)

train_df = train_df.fillna(-1)
#test_df = test_df.fillna(-1)

In [13]:
#save the output
train_X, test_X = train_df[features].as_matrix(), test_df[features].as_matrix()

preds, model = runXGB(train_X, train_y, test_X,\
feature_names=features,
num_rounds = 3500, eta = 0.02,max_depth = 4,verbose_eval=100)

out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("xgb_beta1point45-0.02-leakage.csv", index=False)

In [ ]:
interest_levels = ['low', 'medium', 'high']

tau = {
    'low': 0.69195995, 
    'medium': 0.23108864,
    'high': 0.07695141, 
}

def correct(df):
    y = df[interest_levels].mean()
    a = [tau[k] / y[k]  for k in interest_levels]
    print a

    def f(p):
        for k in range(len(interest_levels)):
            p[k] *= a[k]
        return p / p.sum()

    df_correct = df.copy()
    df_correct[interest_levels] = df_correct[interest_levels].apply(f, axis=1)

    y = df_correct[interest_levels].mean()
    a = [tau[k] / y[k]  for k in interest_levels]
    print a

    return df_correct

In [ ]:
out_df = correct(out_df)


In [ ]:
out_df.to_csv('xgb_beta1point45-0.02-correct.csv',index=False)

In [ ]:
#save the model
xgbmodel = 'xgb145.model'
modelFileObject = open(xgbmodel,'wb') 
pickle.dump(model,modelFileObject)   
modelFileObject.close()